In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import openai


In [2]:
load_dotenv()

openai.api_key = os.getenv('API_KEY')

In [3]:
column_names = ['Varenummer', 'Kapittel', 'Beskrivelse']  # Example column names, adjust as needed
df = pd.read_csv("Bku.csv", sep=';', header=None, names=column_names).loc[:100]

df.head()

,Varenummer,Kapittel,Beskrivelse
0,10059010,Grovfor til storfe,"Hele maisplanter, med maiskolbe, som er finsni..."
1,10062090,Fullkorns basmatiris 500 gram,"Varen er ukokt, avskallet ""brown"" fullkorns ba..."
2,10062090,Ris,"Varen er avlange, avskallede rå riskorn av tha..."
3,10062090,Ris,"Varen ukokt, avskallet, upolert, ""svart jasmin..."
4,10062090,Ris,"Varen langkornet, avskallet, upolert ris, som ..."


In [4]:
print(df.columns)
print(f"Number of columns: {len(df.columns)}")


Index(['Varenummer', 'Kapittel', 'Beskrivelse'], dtype='object')
Number of columns: 3


In [5]:
df.dropna( inplace=True )
df.shape

(101, 3)

In [6]:

df["VareBeskrivelse"] = df["Kapittel"] + df["Beskrivelse"]
df.head(2)


,Varenummer,Kapittel,Beskrivelse,VareBeskrivelse
0,10059010,Grovfor til storfe,"Hele maisplanter, med maiskolbe, som er finsni...","Grovfor til storfeHele maisplanter, med maisko..."
1,10062090,Fullkorns basmatiris 500 gram,"Varen er ukokt, avskallet ""brown"" fullkorns ba...","Fullkorns basmatiris 500 gramVaren er ukokt, a..."


In [7]:
for i in range(3):
    print(df["Varenummer"].iloc[i])
    print(df["Kapittel"].iloc[i])
    print(df["Beskrivelse"].iloc[i])
    
    


10059010
Grovfor til storfe
Hele maisplanter, med maiskolbe, som er finsnittet og ensilert/konservert uten tilsetninger i store plansiloer med plast på toppen. Dette blir lastet opp i lastebil og kjørt i bulk til Norge eller blir presset i runde baller med plast på utsiden. Brukes som grovfor til storfe.
10062090
Fullkorns basmatiris 500 gram
Varen er ukokt, avskallet "brown" fullkorns basmatiris uten andre tilsetninger. Koketid ca. 25 minutt. Pakningsstørrelse: 500g.
10062090
Ris
Varen er avlange, avskallede rå riskorn av thailandsk brun ris. Koketid 45 minutter. Pakket i plastpose à 1 kg.


## Henter text embedding der vi bruker openai text-embedding-ada-002 MODEL


In [8]:
import openai

#OpenAI API-nøkkelen
openai.api_key = os.getenv('API_KEY')
#et skrip som vektoriserer dataten og lagrer det. Det blir gjenbrukbart. ! 1 skript: Vektoriser dataen. 2: Lage nytt skriv
#
def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    response = openai.Embedding.create(input=[text], model=model)
    return response['data'][0]['embedding']


In [9]:
sample_embedding = get_embedding("ribbe")

In [10]:
len(sample_embedding)

1536

In [11]:
df["NavnVareBeskrivelse"] = df["VareBeskrivelse"].apply(lambda x: get_embedding(x, model = 'text-embedding-ada-002'))
df.to_csv("BeskrivelseEmbedded.csv")

## Legge til den nye embedded dataen i Elastic search indeksen


In [12]:
from elasticsearch import Elasticsearch

es = Elasticsearch(
    "https://localhost:9200",
    basic_auth = ("elastic", "ElasticSear"),
    ca_certs = "/Users/yousra/TollEtatenFinal/elasticsearch-8.12.1/config/certs/http_ca.crt"
)
es.ping()

False

In [59]:
from indexmapping import indexmapping
import numpy as pd

ImportError: cannot import name 'indexmappingJson' from 'indexmappingJson' (/Users/yousra/csvFilTollEtaten/indexmappingJson.py)

In [51]:
# es.indices.create(index = "vareprodukter", mappings = indexmapping)



In [56]:
import pandas as pd
import numpy as np  # Dette bør også inkluderes siden du ser ut til å bruke np.array senere

embedding_df = pd.read_csv("BeskrivelseEmbedded.csv", index_col = 0)
embedding_df['NavnVareBeskrivelse'] = embedding_df.NavnVareBeskrivelse.apply(eval).apply(np.array)
#Konverter det til en array format


In [57]:
docs = embedding_df.to_dict("records")
docs[:5]

[{'versjon': 1.5,
  'avsnitt': "{'type': 'avsnitt', 'id': '01', 'beskrivelse': 'Levende dyr; animalske produkter', 'kapitler': [{'type': 'kapittel', 'id': '01', 'beskrivelse': 'Levende dyr', 'inndelinger': [{'type': 'posisjon', 'id': '0101', 'beskrivelse': 'Hester, esler, muldyr og mulesler, levende.', 'oppdelinger': [{'type': 'underposisjon', 'beskrivelse': 'hester:', 'hsNummer': '01012', 'oppdelinger': [{'type': 'vare', 'id': '01012100', 'hsNummer': '010121', 'vareslag': 'til avl', 'blv': False, 'henvisninger': ['TNS 001']}, {'type': 'underposisjon', 'beskrivelse': 'andre:', 'hsNummer': '010129', 'oppdelinger': [{'type': 'vare', 'id': '01012902', 'hsNummer': '010129', 'vareslag': 'av vekt under 133 kg', 'blv': False}, {'type': 'vare', 'id': '01012908', 'hsNummer': '010129', 'vareslag': 'ellers', 'blv': False}]}]}, {'type': 'vare', 'id': '01013000', 'hsNummer': '010130', 'vareslag': 'esler', 'blv': False}, {'type': 'vare', 'id': '01019000', 'hsNummer': '010190', 'vareslag': 'andre', '

In [15]:
for doc in docs:
    try:
        es.index(index = "vareprodukter", document = doc, id = doc["Varenummer"])
    except Exception as e:
        print(e)

In [16]:
es.count(index = "vareprodukter")

ObjectApiResponse({'count': 22, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## Søk etter data fra indekset


In [17]:
input_keyword = "ukokt ris" 
#Her konverteres søketeksten til inn til vector
vector_of_input_keyword = get_embedding(input_keyword)

In [19]:
#Nå filtrerer vi de semantiske resultatene.

q1 = {
    "knn": {
        "field" : "NavnVareBeskrivelse", #Sender query vector til søkefeltet også til vector
        "query_vector" : vector_of_input_keyword,
        "k" : 10, #Hvor mange resultater vi ønsker
        "num_candidates" : 1000 #hvor mange kandidter som kommer til å blir lagt til i algoritmen.
    },

    "_source": ["Varenummer", "Kapittel", "Beskrivelse"]
} 


#Eventuelle filter

#filter_query = {}


## PRøver å filtrere her fra search query

In [20]:
kapittel_list = embedding_df["Kapittel"].drop_duplicates().to_list()
print(kapittel_list)
varenummer_list = embedding_df["Varenummer"].drop_duplicates().to_list()
print(varenummer_list)
beskrivelse_list = embedding_df["Beskrivelse"].drop_duplicates().to_list()

['Grovfor til storfe', 'Fullkorns basmatiris 500 gram', 'Ris', 'Ukokt ris', 'Basmatiris 500 gram', 'Ris/ Paella sett', 'ris', 'Ris ukokt', 'Ris (Jasmin)', 'Ris / Paella sett', 'Polert ris', 'Middagssett - risrett', 'Indisk middagssett - risrett', 'korn', 'Quinoa - tørket', 'Quinoa', 'mel', 'Melblanding', 'Hvetemel', 'Finmalt spelt/hvetemel', 'Finmalt hvete/speltmel', 'finmalt hvitt hvete/spelt mel', 'Finmalt hvetemel', 'Finmalt mel', 'Mel', 'Hvetemel, finmalt', 'Maismel', 'rismel', 'Rismel', 'Bokhvete mel', 'Hirsemel, finmalt', 'Glutenfritt håndtverksmjøl, teff', 'hirse mel Korakan', 'gryn', 'Semulegryn', 'Maismel, grovt', 'Havremel', 'havregryn', 'Havregryn', 'Quinoaflak', 'Valset mais flak', 'Speltflak', 'Hirseflak', 'Bokhvete', 'Avskallet korn av hvete', 'Potetmos', 'Potetpulver, lufttørket', 'Potetpulver,tørket', 'Potetflak', 'Søtpotetpulver', 'Pulver av solbær', 'Aprikospulver', 'Pulver', 'Sitrus pulver', 'Nypepulver', 'Sitrusfiber', 'Aronia pulver', 'Kokosmel', 'Pulver av Baobab 

In [21]:
input_keyword = "ukokt ris"
vector_of_input_keyword = get_embedding(input_keyword)

## her skal vi gi prompt spørring

In [22]:
prompt = f"""dataen min ligger nå i elastic search med varenummer, kapitler, beskrivelse etc.
Varenummerene vi har er{varenummer_list}. Kapitler vi har er {kapittel_list} Vi har også beskrivelser {beskrivelse_list}. Baser på søkefeltet, fi meg en json output som følger: "
{{
"varenummer": "skal egt skrive noe her. Gi not found etc hvos spørring et eller annet"
"kapittel":"Skal egt skrive noe her"
"Beskrivelse": "her og" 
}}

users query : {input_keyword}
"""

prompt

'dataen min ligger nå i elastic search med varenummer, kapitler, beskrivelse etc.\nVarenummerene vi har er[10059010, 10062090, 10063080, 10064080, 10081090, 10085000, 11010000, 11022090, 11029002, 11029009, 11031100, 11031390, 11031990, 11041200, 11041900, 11042902, 11042909, 11051000, 11052000, 11061090, 11062000, 11063090]. Kapitler vi har er [\'Grovfor til storfe\', \'Fullkorns basmatiris 500 gram\', \'Ris\', \'Ukokt ris\', \'Basmatiris 500 gram\', \'Ris/ Paella sett\', \'ris\', \'Ris ukokt\', \'Ris (Jasmin)\', \'Ris / Paella sett\', \'Polert ris\', \'Middagssett - risrett\', \'Indisk middagssett - risrett\', \'korn\', \'Quinoa - tørket\', \'Quinoa\', \'mel\', \'Melblanding\', \'Hvetemel\', \'Finmalt spelt/hvetemel\', \'Finmalt hvete/speltmel\', \'finmalt hvitt hvete/spelt mel\', \'Finmalt hvetemel\', \'Finmalt mel\', \'Mel\', \'Hvetemel, finmalt\', \'Maismel\', \'rismel\', \'Rismel\', \'Bokhvete mel\', \'Hirsemel, finmalt\', \'Glutenfritt håndtverksmjøl, teff\', \'hirse mel Korakan

In [28]:
context_prompt = """Du er en Tolltariffside i Norge som profesjonelt klassifiserer varer.
Din rolle blir å gi meg riktig varenummer utifra det jeg søker etter. 
Jeg er en importør som skal søke etter varer i din nettside og det er kritisk at jeg får korrekt varenummer ettersom dette skal importeres fra utlandet til Norge.
Du skal først gi meg varens posisjon ("heading" i HS-nomenklaturen") på fire sifre,
så skal du gi underposisjonen("subheading" i HS-nomenklaturen). Du skal gi meg de 6 første sifrene som er internasjonale uten punktum og mellomrom: [15042011].
Deretter gir du meg det 8-sifrede formatet, der de to siste sifrene er det som tilsvarer det norske, altså et tolltariffnummer. Disse ekstra to sifrene reflekterer nasjonale behov for klassifisering av varer.
Du skal legge til de to sifrene ettersom de fleste land legger til ytterligere sifre for å gjøre klassifiseringen mer spesifikk
De to nasjonale sifrene benyttes når det er behov for å skille ut varer med forskjellige tollavgiftsatset,, særavgifter, 
innførsels- og utførselsrestriksjoner, og til statistikk over utenrikshandelen.
Gi meg minst tre eksempler som kan være aktuelle for varebeskrivelsen.

Eksempel:
- av svin 
- - - "Bacon crisp"
Posisjon (4 siffer): 1602
Underposisjon (6 siffer): 160249
Nasjonalt varenummer (8 siffer): 16024910

Denne nettsiden er rettet mot alle individer som skal eksportere varer fra utlandet 
til Norge som ønsker å ha klar oversikt over tollavgiftsatset, særavgifter, innførsels- og utførselsrestriksjoner,
 og til statistikk over utenrikshandelen.

Varebeskrivelse:
"""

In [23]:
"""response = client.chat.completions.create(
    model = "gbt-3.5-turbo-1106",
    response_format = { "type": "json_object"},
    messages = [
        {"role": "system", "content": "En ganske hjelpsom assistent lagetfor å gi output i JSON format"},
        {"role": "user", "content": prompt}
    ]
)

response"""

'response = client.chat.completions.create(\n    model = "gbt-3.5-turbo-1106",\n    response_format = { "type": "json_object"},\n    messages = [\n        {"role": "system", "content": "En ganske hjelpsom assistent lagetfor å gi output i JSON format"},\n        {"role": "user", "content": prompt}\n    ]\n)\n\nresponse'

In [29]:

# Her definerer vi en funksjon for å skape chat-completions
def create_chat_completion(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Du er en svært intelligent assistent laget for å gi output i JSON format."},
            {"role": "user", "content": context_prompt + prompt}
        ]
    )
    return response

# Anta at `prompt` er brukerens spørsmål eller kommando
prompt = "ukokt ris"
response = create_chat_completion(prompt)
print(response)


{
  "id": "chatcmpl-8xcq1dIrrfdbM3sxn8832uFO3yrh7",
  "object": "chat.completion",
  "created": 1709221337,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "### Eksempel 1:\n- Varebeskrivelse: Ukokt ris\n- Posisjon (4 siffer): 1006\n- Underposisjon (6 siffer): 100610\n- Nasjonalt varenummer (8 siffer): 10061000\n\n### Eksempel 2:\n- Varebeskrivelse: T\u00f8rket frukt, ikke annensteds oppf\u00f8rt eller inkludert\n- Posisjon (4 siffer): 0813\n- Underposisjon (6 siffer): 081340\n- Nasjonalt varenummer (8 siffer): 08134088\n\n### Eksempel 3:\n- Varebeskrivelse: Olivenolje\n- Posisjon (4 siffer): 1509\n- Underposisjon (6 siffer): 150910\n- Nasjonalt varenummer (8 siffer): 15091000\n\nDisse vareeksemplene gir deg den internasjonale posisjonen, underposisjonen og det norske tolltariffnummeret for \u00e5 kunne importere varene til Norge korrekt."
      },
      "logprobs": null,
      "finish_reason": "

In [25]:
response.choices[0].message.content

'For å koke ris, følg disse enkle stegene:\n\n1. Skyll risen grundig under kaldt vann i en sil eller dørslag til vannet er klart. Dette fjerner overflødig stivelse og bidrar til å få en løs og fluffy konsistens.\n\n2. Mål opp ønsket mengde ris og vann. For de fleste typer ris brukes vanligvis en 2:1 vann-til-ris ratio.\n\n3. Ha risen og vannet i en kjele og tilsett en klype salt, om ønskelig. Rør forsiktig sammen.\n\n4. Sett kjelen på komfyren og bring vannet til å koke over middels varme.\n\n5. Reduser varmen til lavt og sett på lokket. La risen småkoke i ca. 15-20 minutter, eller til all væsken er absorbert og risen er mør og luftig.\n\n6. Fjern kjelen fra varmen og la den stå i ytterligere 5-10 minutter med lokket på. Dette hjelper til med å dampe risen ferdig og gjøre den mer fluffig.\n\n7. Når risen er ferdig, rør forsiktig med en gaffel for å løsne den og server varm.\n\nHåper dette hjelper!'

In [27]:
import json
filter_map = json.loads(response.choices[0].message.content)
filter_map

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Generere JSON ouput basert på brukerens søk

In [ ]:
"""#`response` er svaret du får fra OpenAI
#konverterer svaret til en JSON-struktur
def convert_to_json(response):
    #trekke ut informasjon fra svaret
    #tilpasse basert på strukturen til svaret man får fra OpenAI
    data = response['choices'][0]['message']['content']
    json_output = {
        "varenummer": "10059010",  # Erstatt dette med faktiske verdier
        "kapittel": "Grovfor til storfe",  # Erstatt dette med faktiske verdier
        "beskrivelse": "Hele maisplanter, med maiskolbe, som er finsnittet og ensilert..."  # Erstatt dette med faktiske verdier
    }
    return json_output

# Bruk denne funksjonen til å konvertere svaret til JSON
json_response = convert_to_json(response)
print(json.dumps(json_response, indent=2, ensure_ascii=False))
 """

In [ ]:
"""q1 = {
    "knn": {
        "field" : "NavnVareBeskrivelse", #Sender query vector til søkefeltet også til vector
        "query_vector" : vector_of_input_keyword,
        "k" : 10, #Hvor mange resultater vi ønsker
        "num_candidates" : 1000 #hvor mange kandidter som kommer til å blir lagt til i algoritmen.
    },

    "_source": ["Varenummer", "Kapittel", "Beskrivelse"]
} """

In [10]:
from flask import Flask, request, jsonify
app = Flask(__name__)

@app.route('/get-explanation', methods=['POST'])
def get_explanation():
    data = request.json
    input_text = data['input_text']
# Anta at du har en funksjon som behandler input_text og returnerer en forklaring
    explanation = process_input_and_generate_explanation(input_text)
    return jsonify({"explanation": explanation})

if name == 'main':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
Traceback (most recent call last):
  File "/Users/yousra/csvFilTollEtaten/venv/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/yousra/csvFilTollEtaten/venv/lib/python3.9/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/Users/yousra/csvFilTollEtaten/venv/lib/python3.9/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/Users/yousra/csvFilTollEtaten/venv/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/Users/yousra/csvFilTollEtaten/venv/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 331, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/Users/yousra/csvFilTollEtaten/venv/lib/pyth

SystemExit: 1